In [1]:
import requests
import os
import json
import pprint
from time import sleep
import sched
from datetime import datetime
from time import sleep
import time
from IPython.display import clear_output

In [2]:
def clear():
    os.system('cls')

In [3]:
def reqAPI():
    try:
        r = requests.get('https://www.cryptopia.co.nz/api/GetMarkets/BTC')
        coins = json.loads(r.text)['Data']
        if coins is not None:
            return coins #lista de diccionarios 
    except ValueError:
        print('Error al hacer el request, intentando nuevamente')
        reqAPI()

In [4]:
def refreshData():
    #API REQUEST
    lenDic, coc, last, volC = 0, 0, 0, 0
    ask, bid, spread = 0, 0, 0
    old, recent = 0, 0
    coins = reqAPI()
    while coins is None:
        coins = reqAPI()
        print(type(coins))
    for y in coins:
        subDic = {}
        changeDic = {}
        name = y['Label']
        if name in dic:
            lenDic = len(dic[name])
        else:
            dic[name] = {}
        ts = datetime.utcnow().strftime('%Y-%m-%d %H:%M:%S')
        if y['BidPrice'] != 0:
            spread = ((y['AskPrice']/y['BidPrice'])-1)*100
            spread = round(spread, 2)
        changeDic = {'last':y['LastPrice'],
                     'sprd':spread,
                     'chng':y['Change'],
                    'vol': (0, y['Volume'])}
        if lenDic != 0: #Si hay registros de precios
            #Cambio
            recent = [*dic[name]][lenDic-1] #Compara con el más reciente
            last = dic[name][recent]['last']
            coc = round((y['LastPrice']/last)-1, 2)
            #Volumen
            last = dic[name][recent]['vol'][1] #Compara el volumen más reciente
            #Captura de los valores viejo y nuevo
            changeDic['vol'] = (last, y['Volume'])
            if last != 0:
                volC = round((y['Volume']/last)-1, 2)
            elif y['Volume'] == 0:
                volC = 0
            else:
                volC = 1
            changeDic['coc'] = coc
            changeDic['volC'] = volC
            if coc > 0.0 and volC > 0.0: #Si el cambio en precios o en volumen es mayor al 5%
                if not name in toBuy:
                    toBuy[name] = {}
                toBuy[name][ts] = changeDic
        if lenDic == 5: 
            old = [*dic[name]][0] #Registro más viejo
            dic[name].pop(old)
        #Inserta el nuevo
        dic[name][ts] = changeDic

In [5]:
#Variables
#Diccionarios con los datos
def analiza(n):
    pp = pprint.PrettyPrinter(indent=1)
    i = 0
    suma = 0
    toBuySize = 0
    while i < n:
        start = time.time()
        refreshData()
        end = time.time()
        suma += end-start
        #clear()
        if toBuySize < len(toBuy): #Si se agregó un nvo item a toBuy
            clear_output()
            pp.pprint(toBuy)
        i += 1
        toBuySize = len(toBuy)
        sleep(1.5)
        
    #Tiempo promedio por respuesta
    print('Tiempo promedio de las ejecuciones: ', suma/n)

In [6]:
#Coin Analisis
#pp.pprint(dic)
#pp.pprint(dic['$$$/BTC'])
dic, toBuy, changeDic = {}, {}, {}
n = input('Ingresar número de pruebas a ejecutar ')
if type(n) is str:
    n = int(n)
    if n > 0:
        analiza(n)
else:
    print('Ingresa un número mayor a 0')

{'ARC/BTC': {'2018-11-15 18:54:37': {'chng': -0.57,
                                     'coc': 0.05,
                                     'last': 1.74e-06,
                                     'sprd': 1.72,
                                     'vol': (18916.33175573, 20356.59686409),
                                     'volC': 0.08},
             '2018-11-15 18:55:08': {'chng': 1.14,
                                     'coc': 0.02,
                                     'last': 1.77e-06,
                                     'sprd': 1.72,
                                     'vol': (20356.59686409, 34427.97002569),
                                     'volC': 0.69}},
 'BNX/BTC': {'2018-11-15 18:55:39': {'chng': 0.0,
                                     'coc': 0.2,
                                     'last': 6e-08,
                                     'sprd': 20.0,
                                     'vol': (0.0, 16633.39986693),
                                     'volC': 1}},
 'CAN

TypeError: 'NoneType' object is not subscriptable